In [1]:
from templates import *
from templates_cls import *
from experiment_classifier import ClsModel
import matplotlib.pyplot as plt

In [2]:
device = 'cuda:1'
conf = ffhq128_autoenc_w_classifier()
conf.name = 'ffhq128_autoenc_w_classifier_weightedLoss_step_BN'
# print(conf.name)
model = LitModel(conf)
state = torch.load(f'checkpoints/{conf.name}/last.ckpt', map_location='cpu')
model.load_state_dict(state['state_dict'], strict=False)
model.ema_model.eval()
model.ema_model.to(device);

Global seed set to 0
Using cache found in /users/biocomp/mahanta/.cache/torch/hub/pytorch_vision_v0.10.0


Model params: 124.97 M


In [5]:
print(conf.name)

ffhq128_autoenc_w_classifier_weightedLoss_step_BN


In [3]:
print(model)

LitModel(
  (model): BeatGANsAutoencModel(
    (time_embed): TimeStyleSeperateEmbed(
      (time_embed): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
      )
      (style): Identity()
    )
    (input_blocks): ModuleList(
      (0): TimestepEmbedSequential(
        (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1-2): 2 x TimestepEmbedSequential(
        (0): ResBlock(
          (in_layers): Sequential(
            (0): GroupNorm32(32, 128, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (h_upd): Identity()
          (x_upd): Identity()
          (emb_layers): Sequential(
            (0): SiLU()
            (1): Linear(in_features=512, out_features=256, bias=True)
          )
          (cond_emb_layers): Sequential(
          

In [9]:
model.model.classifier_component

Classifier_Component(
  (mobile_net): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, ke

In [12]:
cls_conf = ffhq128_autoenc_cls()
cls_conf.pretrain = PretrainConfig(
        '130M',
        f'checkpoints/{ffhq128_autoenc_w_classifier().name}/last.ckpt',
    )
cls_celeb = ClsModel(cls_conf)



state = torch.load(f'checkpoints/{cls_conf.name}/last.ckpt',
                    map_location='cpu')
print('latent step:', state['global_step'])
cls_celeb.load_state_dict(state['state_dict'], strict=False);
cls_celeb.to(device)

Global seed set to 0
Using cache found in /users/biocomp/mahanta/.cache/torch/hub/pytorch_vision_v0.10.0


loading pretrain ... 130M
step: 2145


RuntimeError: Error(s) in loading state_dict for ClsModel:
	size mismatch for model.classifier_component.linear_projection.0.weight: copying a param with shape torch.Size([512, 514]) from checkpoint, the shape in current model is torch.Size([514]).
	size mismatch for model.classifier_component.linear_projection.0.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([514]).
	size mismatch for ema_model.classifier_component.linear_projection.0.weight: copying a param with shape torch.Size([512, 514]) from checkpoint, the shape in current model is torch.Size([514]).
	size mismatch for ema_model.classifier_component.linear_projection.0.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([514]).

In [ ]:
data = ImageDataset('imgs_align', image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False)
batch = data[1]['img'][None]

In [ ]:
cond = model.encode(batch.to(device))


xT = model.encode_stochastic(batch.to(device), cond, T=250)